# SQLAlchemy Core

## Schema and Types

In [1]:
from datetime import datetime

from sqlalchemy import DateTime
from sqlalchemy import MetaData
from sqlalchemy import Table, Column, Integer, String, Boolean, Float

In [2]:
metadata = MetaData()

In [3]:
cookies = Table('cookies', metadata,
                Column('cookie_id', Integer(), primary_key=True),
                Column('cookie_name', String(50), index=True),
                Column('cookie_recipe_url', String(255)),
                Column('cookie_sku', String(55)),
                Column('quantity', Integer()),
                Column('unit_cost', Float(12))
                )

In [4]:
users = Table('users', metadata,
              Column('user_id', Integer(), primary_key=True),
              Column('customer_number', Integer(), autoincrement=True),
              Column('username', String(15), nullable=False, unique=True),
              Column('email_address', String(255), nullable=False),
              Column('phone', String(20), nullable=False),
              Column('password', String(25), nullable=False),
              Column('created_on', DateTime(), default=datetime.now),
              Column('updated_on', DateTime(), default=datetime.now, onupdate=datetime.now)
              )

In [5]:
from sqlalchemy import ForeignKey

orders = Table('orders', metadata,
               Column('order_id', Integer(), primary_key=True),
               Column('user_id', ForeignKey('users.user_id')),
               Column('shipped', Boolean(), default=False)
               )

In [6]:
line_items = Table('line_items', metadata,
                   Column('line_items_id', Integer(), primary_key=True),
                   Column('order_id', ForeignKey('orders.order_id')),
                   Column('cookie_id', ForeignKey('cookies.cookie_id')),
                   Column('quantity', Integer()),
                   Column('extended_cost', Float(12))
                   )

In [7]:
from sqlalchemy import create_engine

engine = create_engine('sqlite:///:memory:')
connection = engine.connect()
metadata.create_all(engine)

## Working with Data via SQLAlchemy Core

### Insert

In [8]:
from sqlalchemy import insert

In [9]:
ins = insert(cookies).values(
    cookie_name="chocolate chip",
    cookie_recipe_url="https://some.aweso.me/cookie/recipe.html",
    cookie_sku="CC01",
    quantity="12",
    unit_cost="0.50"
)

In [10]:
ins.compile().params

{'cookie_name': 'chocolate chip',
 'cookie_recipe_url': 'https://some.aweso.me/cookie/recipe.html',
 'cookie_sku': 'CC01',
 'quantity': '12',
 'unit_cost': '0.50'}

In [11]:
result = connection.execute(ins)
result.inserted_primary_key

(1,)

In [12]:
ins = cookies.insert()
result = connection.execute(
    ins,
    cookie_name='dark chocolate chip',
    cookie_recipe_url='https://some.aweso.me/cookie/recipe_dark.html',
    cookie_sku='CC02',
    quantity='1',
    unit_cost='0.75'
)

In [13]:
result.inserted_primary_key

(2,)

In [14]:
inventory_list = [
    {
        'cookie_name': 'peanut butter',
        'cookie_recipe_url': 'https://some.aweso.me/cookie/peanut.html',
        'cookie_sku': 'PB01',
        'quantity': '24',
        'unit_cost': '0.25'
    },
    {
        'cookie_name': 'oatmeal raisin',
        'cookie_recipe_url': 'https://some.okay.me/cookie/raisin.html',
        'cookie_sku': 'EWW01',
        'quantity': '100',
        'unit_cost': '1.00'
    }
]

In [15]:
result = connection.execute(ins, inventory_list)

### Querying

In [16]:
from sqlalchemy.sql import select

s = select(cookies)

In [17]:
rp = connection.execute(s)
results = rp.fetchall()
results

[(1, 'chocolate chip', 'https://some.aweso.me/cookie/recipe.html', 'CC01', 12, 0.5),
 (2, 'dark chocolate chip', 'https://some.aweso.me/cookie/recipe_dark.html', 'CC02', 1, 0.75),
 (3, 'peanut butter', 'https://some.aweso.me/cookie/peanut.html', 'PB01', 24, 0.25),
 (4, 'oatmeal raisin', 'https://some.okay.me/cookie/raisin.html', 'EWW01', 100, 1.0)]

In [18]:
first_row = results[0]
print(first_row[1])
print(first_row.cookie_name)
print(first_row[cookies.c.cookie_name])

chocolate chip
chocolate chip
chocolate chip


In [19]:
rp = connection.execute(s)
for record in rp:
    print(record.cookie_name)

chocolate chip
dark chocolate chip
peanut butter
oatmeal raisin


In [20]:
s = select(cookies.c.cookie_name, cookies.c.quantity)
rp = connection.execute(s)
print(rp.keys())
result = rp.first()
print(result)

RMKeyView(['cookie_name', 'quantity'])
('chocolate chip', 12)


In [21]:
s = select(cookies.c.cookie_name, cookies.c.quantity)
s = s.order_by(cookies.c.quantity)
rp = connection.execute(s)
for cookie in rp:
    print(f'{cookie.quantity} - {cookie.cookie_name}')

1 - dark chocolate chip
12 - chocolate chip
24 - peanut butter
100 - oatmeal raisin


In [22]:
from sqlalchemy import desc

s = select(cookies.c.cookie_name, cookies.c.quantity)
s = s.order_by(desc(cookies.c.quantity))
rp = connection.execute(s)
results = rp.fetchall()
results

[('oatmeal raisin', 100),
 ('peanut butter', 24),
 ('chocolate chip', 12),
 ('dark chocolate chip', 1)]

In [23]:
s = select(cookies.c.cookie_name, cookies.c.quantity)
s = s.order_by(cookies.c.quantity)
s = s.limit(2)
# s = select(cookies.c.cookie_name, cookies.c.quantity).order_by(cookies.c.quantity).limit(2)
rp = connection.execute(s)
print([result.cookie_name for result in rp])

['dark chocolate chip', 'chocolate chip']


In [24]:
from sqlalchemy.sql import func

s = select(func.sum(cookies.c.quantity))
rp = connection.execute(s)
print(rp.scalar())

137


In [25]:
s = select(func.count(cookies.c.cookie_name))
rp = connection.execute(s)
record = rp.first()
print(record.keys())
print(record.count_1)

RMKeyView(['count_1'])
4


In [26]:
s = select(func.count(cookies.c.cookie_name).label('inventory_count'))
rp = connection.execute(s)
record = rp.first()
print(record.keys())
print(record.inventory_count)

RMKeyView(['inventory_count'])
4


In [27]:
s = select(cookies).where(cookies.c.cookie_name == 'chocolate chip')
rp = connection.execute(s)
record = rp.first()
print(record)

(1, 'chocolate chip', 'https://some.aweso.me/cookie/recipe.html', 'CC01', 12, 0.5)


In [28]:
s = select(cookies).where(cookies.c.cookie_name.like('%chocolate%'))
rp = connection.execute(s)
for record in rp.fetchall():
    print(record.cookie_name)

chocolate chip
dark chocolate chip


In [29]:
s = select(cookies.c.cookie_name, 'SKU-' + cookies.c.cookie_sku)
for row in connection.execute(s):
    print(row)

('chocolate chip', 'SKU-CC01')
('dark chocolate chip', 'SKU-CC02')
('peanut butter', 'SKU-PB01')
('oatmeal raisin', 'SKU-EWW01')


### Cast

In [30]:
from sqlalchemy import cast

s = select(cookies.c.cookie_name, cast((cookies.c.quantity * cookies.c.unit_cost), Float(12)).label('inv_cost'))
for row in connection.execute(s):
    print(f'{row.cookie_name} - {row.inv_cost}')

chocolate chip - 6.0
dark chocolate chip - 0.75
peanut butter - 6.0
oatmeal raisin - 100.0


### Boolean

In [31]:
from sqlalchemy import and_, or_

s = select(cookies).where(
    and_(
        cookies.c.quantity > 23,
        cookies.c.unit_cost < 0.40
    )
)

for row in connection.execute(s):
    print(row.cookie_name)

peanut butter


In [32]:
s = select(cookies).where(
    or_(
        cookies.c.quantity.between(10, 50),
        cookies.c.cookie_name.contains('chip')
    )
)

for row in connection.execute(s):
    print(row.cookie_name)

chocolate chip
dark chocolate chip
peanut butter


### Update

In [33]:
from sqlalchemy import update

u = update(cookies).where(cookies.c.cookie_name == "chocolate chip")
u = u.values(quantity=(cookies.c.quantity + 120))
result = connection.execute(u)
print(result.rowcount)

1


In [34]:
s = select(cookies).where(cookies.c.cookie_name == "chocolate chip")
result = connection.execute(s).first()
for key in result.keys():
    print(f'{key:>20}: {result[key]}')

           cookie_id: 1
         cookie_name: chocolate chip
   cookie_recipe_url: https://some.aweso.me/cookie/recipe.html
          cookie_sku: CC01
            quantity: 132
           unit_cost: 0.5


In [35]:
from sqlalchemy import delete

u = delete(cookies).where(cookies.c.cookie_name == "dark chocolate chip")
result = connection.execute(u)
print(result.rowcount)

1


In [36]:
s = select(cookies).where(cookies.c.cookie_name == "dark chocolate chip")
result = connection.execute(s).fetchall()
print(len(result))

0


### Join

In [37]:
customer_list = [
    {
        'username': 'cookiemon',
        'email_address': 'mon@cookie.com',
        'phone': '111-111-1111',
        'password': 'password'
    },
    {
        'username': 'cakeeater',
        'email_address': 'cakeeater@cake.com',
        'phone': '222-222-2222',
        'password': 'password'
    },
    {
        'username': 'pieguy',
        'email_address': 'guy@pie.com',
        'phone': '333-333-3333',
        'password': 'password'
    }
]

# ins = users.insert()
# result = connection.execute(ins, customer_list)
ins = insert(users).values(customer_list)
result = connection.execute(ins)
print(result)

In [38]:
s = select(users)
result = connection.execute(s)
print(result.fetchall())

[(1, None, 'cookiemon', 'mon@cookie.com', '111-111-1111', 'password', datetime.datetime(2022, 7, 27, 13, 32, 3, 746751), datetime.datetime(2022, 7, 27, 13, 32, 3, 746751)), (2, None, 'cakeeater', 'cakeeater@cake.com', '222-222-2222', 'password', datetime.datetime(2022, 7, 27, 13, 32, 3, 746751), datetime.datetime(2022, 7, 27, 13, 32, 3, 746751)), (3, None, 'pieguy', 'guy@pie.com', '333-333-3333', 'password', datetime.datetime(2022, 7, 27, 13, 32, 3, 746751), datetime.datetime(2022, 7, 27, 13, 32, 3, 746751))]


In [39]:
ins = insert(orders).values(user_id=1, order_id=1)
result = connection.execute(ins)
print(result)

In [40]:
s = select(orders)
result = connection.execute(s)
print(result.fetchall())

[(1, 1, False)]


In [42]:
order_items = [
    {
        'order_id': 1,
        'cookie_id': 1,
        'quantity': 2,
        'extended_cost': 1.00
    },
    {
        'order_id': 1,
        'cookie_id': 3,
        'quantity': 12,
        'extended_cost': 3.00
    }
]
ins = insert(line_items).values(order_items)
result = connection.execute(ins)
print(result)

In [43]:
ins = insert(orders).values(user_id=2, order_id=2)
result = connection.execute(ins)

In [44]:
s = select(orders)
result = connection.execute(s)
print(result.fetchall())

[(1, 1, False), (2, 2, False)]


In [45]:
order_items = [
    {
        'order_id': 2,
        'cookie_id': 1,
        'quantity': 24,
        'extended_cost': 12.00
    },
    {
        'order_id': 2,
        'cookie_id': 4,
        'quantity': 6,
        'extended_cost': 6.00
    }
]
ins = insert(line_items).values(order_items)
result = connection.execute(ins)
print(result)

In [46]:
s = select(line_items)
result = connection.execute(s)
print(result.fetchall())

[(1, 1, 1, 2, 1.0), (2, 1, 3, 12, 3.0), (3, 2, 1, 24, 12.0), (4, 2, 4, 6, 6.0)]


In [47]:
columns = [orders.c.order_id, users.c.username, users.c.phone, cookies.c.cookie_name, line_items.c.quantity,
           line_items.c.extended_cost]
cookiemon_orders = select(columns)
cookiemon_orders = cookiemon_orders.select_from(orders.join(users).join(line_items).join(cookies)).where(
    users.c.username == 'cookiemon')
result = connection.execute(cookiemon_orders).fetchall()
for row in result:
    print(row)

(1, 'cookiemon', '111-111-1111', 'chocolate chip', 2, 1.0)
(1, 'cookiemon', '111-111-1111', 'peanut butter', 12, 3.0)


In [48]:
columns = [orders.c.order_id, users.c.username, users.c.phone,
           cookies.c.cookie_name, line_items.c.quantity,
           line_items.c.extended_cost]
cookiemon_orders = select(columns)
cookiemon_orders = cookiemon_orders.select_from(orders.join(users).join(
    line_items).join(cookies)).where(users.c.username == 'cookiemon')
result = connection.execute(cookiemon_orders).fetchall()
for row in result:
    print(row)

(1, 'cookiemon', '111-111-1111', 'chocolate chip', 2, 1.0)
(1, 'cookiemon', '111-111-1111', 'peanut butter', 12, 3.0)


In [49]:
columns = [users.c.username, func.count(orders.c.order_id)]
all_orders = select(columns)
all_orders = all_orders.select_from(users.outerjoin(orders))
all_orders = all_orders.group_by(users.c.username)
result = connection.execute(all_orders).fetchall()
for row in result:
    print(row)

('cakeeater', 1)
('cookiemon', 1)
('pieguy', 0)


In [50]:
employee_table = Table('employee', metadata,
                       Column('id', Integer, primary_key=True),
                       Column('manager', None, ForeignKey('employee.id')),
                       Column('name', String(255))
                       )

In [51]:
columns = [users.c.username, func.count(orders.c.order_id)]
all_orders = select(columns)
all_orders = all_orders.select_from(users.outerjoin(orders))
all_orders = all_orders.group_by(users.c.username)
result = connection.execute(all_orders).fetchall()
for row in result:
    print(row)

('cakeeater', 1)
('cookiemon', 1)
('pieguy', 0)


In [52]:
def get_orders_by_customer(cust_name):
    columns = [orders.c.order_id, users.c.username, users.c.phone,
               cookies.c.cookie_name, line_items.c.quantity,
               line_items.c.extended_cost]
    cust_orders = select(columns)
    cust_orders = cust_orders.select_from(users.join(orders).join(line_items).join(cookies))
    cust_orders = cust_orders.where(users.c.username == cust_name)
    result = connection.execute(cust_orders).fetchall()
    return result

In [53]:
get_orders_by_customer('cakeeater')

[(2, 'cakeeater', '222-222-2222', 'chocolate chip', 24, 12.0),
 (2, 'cakeeater', '222-222-2222', 'oatmeal raisin', 6, 6.0)]

In [54]:
def get_orders_by_customer(cust_name, shipped=None, details=False):
    columns = [orders.c.order_id, users.c.username, users.c.phone]
    joins = users.join(orders)
    if details:
        columns.extend([cookies.c.cookie_name, line_items.c.quantity, line_items.c.extended_cost])
        joins = joins.join(line_items).join(cookies)
    cust_orders = select(columns)
    cust_orders = cust_orders.select_from(joins)
    cust_orders = cust_orders.where(users.c.username == cust_name)
    if shipped is not None:
        cust_orders = cust_orders.where(orders.c.shipped == shipped)
    result = connection.execute(cust_orders).fetchall()
    return result

In [55]:
get_orders_by_customer('cakeeater')
get_orders_by_customer('cakeeater', details=True)
get_orders_by_customer('cakeeater', shipped=True)
get_orders_by_customer('cakeeater', shipped=False)
get_orders_by_customer('cakeeater', shipped=False, details=True)

[(2, 'cakeeater', '222-222-2222', 'chocolate chip', 24, 12.0),
 (2, 'cakeeater', '222-222-2222', 'oatmeal raisin', 6, 6.0)]

In [56]:
result = connection.execute("select * from orders").fetchall()
print(result)

[(1, 1, 0), (2, 2, 0)]


In [57]:
from sqlalchemy import text

stmt = select([users]).where(text("username='cookiemon'"))
print(connection.execute(stmt).fetchall())

[(1, None, 'cookiemon', 'mon@cookie.com', '111-111-1111', 'password', datetime.datetime(2022, 7, 27, 13, 32, 3, 746751), datetime.datetime(2022, 7, 27, 13, 32, 3, 746751))]
